In [1]:
import numpy as np
import pandas as pd

In [2]:
dataSet=[['y','y','y'],
        ['y','y','y'],
        ['y','n','n'],
        ['n','y','n'],
        ['n','y','n']]
labels = ['no surfacing','flippers','labels']

In [3]:
data = pd.DataFrame(data = dataSet,columns=labels)
data

,no surfacing,flippers,labels
0,y,y,y
1,y,y,y
2,y,n,n
3,n,y,n
4,n,y,n


In [4]:
all_data_size = data.index.size
all_data_size

5

In [5]:
data['labels']

0    y
1    y
2    n
3    n
4    n
Name: labels, dtype: object

In [6]:
sum_label_y = (data.iloc[:,-1] == 'y').sum()
sum_label_y

2

In [7]:
sum_label_n = (data.iloc[:,-1] == 'n').sum()
sum_label_n

3

In [8]:
H_y_arr = np.array([sum_label_y,sum_label_n]) / all_data_size
H_y_arr

array([0.4, 0.6])

In [9]:
H_y = np.sum(-H_y_arr*np.log2(H_y_arr))
H_y

0.9709505944546686

In [10]:
data_x1_y = data[data['no surfacing'] == 'y']
data_x1_y

,no surfacing,flippers,labels
0,y,y,y
1,y,y,y
2,y,n,n


In [11]:
sum_x1_y = data_x1_y.index.size
sum_x1_y

3

In [12]:
H_x1_y = (data_x1_y.iloc[:,-1] == 'y').sum()
H_x1_y

2

In [13]:
H_x1_n = (data_x1_y.iloc[:,-1] == 'n').sum()
H_x1_n

1

In [14]:
H_x1_arr = np.array([H_x1_y,H_x1_n]) / sum_x1_y
H_x1_arr

array([0.66666667, 0.33333333])

In [15]:
H_x1 = np.sum(-H_x1_arr*np.log2(H_x1_arr))
H_x1

0.9182958340544896

In [16]:
data_x1_n = data[data['no surfacing'] == 'n']
data_x1_n

,no surfacing,flippers,labels
3,n,y,n
4,n,y,n


In [17]:
sum_x1_y_n = data_x1_n.index.size
sum_x1_y_n

2

In [18]:
H_x1_n_y = (data_x1_n.iloc[:,-1] == 'y').sum()
H_x1_n_y

0

In [19]:
H_x1_n_n = (data_x1_n.iloc[:,-1] == 'n').sum()
H_x1_n_n

2

In [20]:
H_x1_n_arr = np.array([H_x1_n_y + 1e-10,H_x1_n_n]) / sum_x1_y_n
H_x1_n_arr

array([5.e-11, 1.e+00])

In [21]:
H_x1_n = np.sum(-H_x1_n_arr * np.log2(H_x1_n_arr))
H_x1_n

1.7109640474436813e-09

In [22]:
joker1 = data[data['no surfacing'] == 'y']
joker1

,no surfacing,flippers,labels
0,y,y,y
1,y,y,y
2,y,n,n


In [23]:
joker1.iloc[:,1:]
joker1.drop(columns='no surfacing')

,flippers,labels
0,y,y
1,y,y
2,n,n


In [24]:
joker2 = data[data['no surfacing'] == 'n']
joker2

,no surfacing,flippers,labels
3,n,y,n
4,n,y,n


In [25]:
joker2.iloc[:,1:]
joker2.drop(columns='no surfacing')

,flippers,labels
3,y,n
4,y,n


In [26]:
pd.unique(data[data['no surfacing'] == 'y'].iloc[:,-1]).size

2

In [27]:
pd.unique(data[data['no surfacing'] == 'n'].iloc[:,-1]).size

1

In [28]:
def calcShannonEnt_labels(data):
    allData_size = data.index.size
    labels = data.iloc[:,-1]
    labels_dict = {}
    H_y = []
    for label in labels:
        if label not in labels_dict:
            labels_dict[label] = 1 
        else:
            labels_dict[label] += 1
        
    for key,value in labels_dict.items():
        H_y.append(value)
    
    H_y_arr = np.array(H_y) /allData_size
    Ent_H_y = np.sum(-H_y_arr * np.log2(H_y_arr))
    
    return Ent_H_y
        

In [29]:
calcShannonEnt_labels(data)

0.9709505944546686

In [30]:
def calcShannonEnt_feture(data):
    
    all_data_size = data.index.size
    fetures_name = data.columns[:-1]
    best_feture_dict = {}
    for feture in fetures_name:
        unique_feture_son = pd.unique(data[feture])
        Ent_sum = 0
        for feture_son in unique_feture_son:
            # data[data['no surfacing']=='y']
            feture_data_son = data[data[feture]==feture_son]
            fenzi_size = feture_data_son.index.size
            # 最重要
            Ent_feture_son = calcShannonEnt_labels(feture_data_son)
            Ent_sum += Ent_feture_son * fenzi_size / all_data_size

        best_feture_dict[feture] = Ent_sum
    
    best_feture = sorted(best_feture_dict.items(),key=lambda z:z[1])
    
    return best_feture[0][0]

In [31]:
best_feture = calcShannonEnt_feture(data)
best_feture

'no surfacing'

In [32]:
def split_Trees(data,best_feture):
    
    unique_bf = pd.unique(data[best_feture])
    Split_result = []
    sum_f = 0
    for feture_son in unique_bf:
        data_son = data[data[best_feture]==feture_son]
        label_size = pd.unique(data_son.iloc[:,-1]).size
        
        if label_size == 1:
            sum_f += data_son.index.size
            Split_result.append([best_feture,feture_son])
        else:
            drop_data_son = data_son.drop(columns=best_feture)
            Split_result.append([best_feture,feture_son,drop_data_son])
    
    return Split_result,sum_f

In [33]:
Split_result,sum_f = split_Trees(data,best_feture)
Split_result,sum_f

([['no surfacing', 'y',   flippers labels
   0        y      y
   1        y      y
   2        n      n], ['no surfacing', 'n']], 2)

In [34]:
def DT(data):
    REST = []
    ALL_data_size = data.index.size
    ALL_data_fetures = data.columns[:-1].size
    sum_data =0
    sum_feture = 0
    while 1:
        # 1.所有样本都已经使用完毕
        if sum_data == ALL_data_size:
            print('所以样本划分完毕')
            return REST
        # 2.所有特征使用完毕；2.1 样本使用完毕,2.2样本为划分完毕
        if sum_feture == ALL_data_fetures:
            if sum_data == ALL_data_size:
                print('样本划分完毕')
                return REST
            else:
                print('特征已经使用完毕,但是样本依然没有完全划分')
                return REST
        best_feture = calcShannonEnt_feture(data)
        sum_feture +=1
        Split_result,sum_f = split_Trees(data,best_feture)
        sum_data += sum_f
        for i in Split_result:
            if len(i) == 2:
                REST.append(i)
            else:
                data = i[-1]

In [35]:
DT(data)

所以样本划分完毕


[['no surfacing', 'n'], ['flippers', 'y'], ['flippers', 'n']]

In [36]:
data.columns[:-1].size

2